In [1]:
import bw2data as bd
import bw2calc as bc
import bw2analyzer as ba
import bw2io as bi
from tqdm import tqdm

In [2]:
bd.__version__, bc.__version__, ba.__version__, bi.__version__

((4, 0, 'DEV18'), (2, 0, 'DEV12'), (0, 11, 4), (0, 9, 'DEV14'))

In [3]:
bi.restore_project_directory("/Users/ajakobs/Documents/data/hybrid-eco-exio/eco-ex-March-2023-aggregated.tar.gz")

Restoring project backup archive - this could take a few minutes...


'eco-ex-March-2023'

In [4]:
bd.projects.set_current("eco-ex-March-2023")

In [5]:
bd.databases

Databases dictionary with 5 object(s):
	biosphere3
	ecoinvent-3.9.1-cutoff
	exiobase-3.8.1-monetary
	exiobase-3.8.1-monetary biosphere
	swiss consumption 1.0

In [6]:
del bd.databases['swiss consumption 1.0']

In [7]:
for db in bd.databases:
    print(db, bd.databases[db]['depends'])

biosphere3 []
ecoinvent-3.9.1-cutoff ['biosphere3', 'exiobase-3.8.1-monetary']
exiobase-3.8.1-monetary biosphere []
exiobase-3.8.1-monetary ['biosphere3']


In [8]:
imp = bi.CSVImporter('/Users/ajakobs/Library/Application Support/Brightway3/CCL-dev.f4b5e8d8/output/lci-swiss-consumption-10.csv')

Extracted 1 worksheets in 0.12 seconds


In [9]:
def fix_wastewater(data):
    OLD = {
        'treatment of wastewater, average, capacity 1.1E10l/year',
        'treatment of wastewater, average, capacity 1.6E8l/year',
        'treatment of wastewater, average, capacity 1E9l/year',
        'treatment of wastewater, average, capacity 4.7E10l/year',
        'treatment of wastewater, average, capacity 5E9l/year',
    }
    NEW = 'treatment of wastewater, average, wastewater treatment'
    
    for ds in data:
        for exc in ds['exchanges']:
            if exc['name'] in OLD:
                exc['name'] = NEW
    return data

In [10]:
imp.strategies.append(fix_wastewater)

In [11]:
%%time
imp.apply_strategies()
imp.match_database(fields=['name', 'location'])
imp.match_database('ecoinvent-3.9.1-cutoff')
imp.match_database('ecoinvent-3.9.1-cutoff', fields=['name', 'unit', 'location'])
imp.match_database('exiobase-3.8.1-monetary', fields=['name', 'location'])
imp.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applying strategy: fix_wastewater
Applied 17 strategies in 18.73 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
314 d

(314, 28223, 0)

In [12]:
imp.write_database()

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Title: Writing activities to SQLite3 database:
  Started: 03/28/2023 21:05:08
  Finished: 03/28/2023 21:05:10
  Total time elapsed: 00:00:02
  CPU %: 95.30
  Memory %: 2.75
Created database: swiss consumption 1.0


In [13]:
bd.databases

Databases dictionary with 5 object(s):
	biosphere3
	ecoinvent-3.9.1-cutoff
	exiobase-3.8.1-monetary
	exiobase-3.8.1-monetary biosphere
	swiss consumption 1.0

## back up for further use

In [14]:
bi.backup_project_directory(project="eco-ex-March-2023")


Creating project backup archive - this could take a few minutes...


In [15]:
# saved here: "/Users/ajakobs/brightway2-project-eco-ex-March-2023-backup.28-March-2023-09-05PM.tar.gz